In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

%matplotlib inline

import math
import numpy as np
import requests
import ipywidgets as widgets
import time
import parmed as pmd
import re

import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path
from scipy import spatial

import threading

NGL_DEF = False
try:
    import nglview as nv
    NGL_DEF = True
except:
    NGL_DEF = False

import hublib.use
%use gromacs-2018.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   

In [ ]:
def writelog(msg, leadchar):
    with open("./logfile.txt", 'a') as fd:
        for line in msg:
            fd.write(leadchar + " " + line + "\n")
            
def runbash(cmd, comment):
    out = !{cmd}
    writelog(["*****************************************************"], '')
    writelog([comment], '# ')
    writelog(["Running bash command:"], '# ')
    writelog([cmd+'\n'], 'bash >> ')
    writelog(out, 'out: ')
    writelog(["*****************************************************"], '')
    return out

In [ ]:
out = runbash("cp ../em/complex.top ./", 'Copying topology file complex.top from em folder')
out = runbash("cp ../em/*.itp ./", 'Copying all itp files from em folder')
out = runbash("cp ../em/em.gro ./", 'Copying em.gro as input')
out = runbash("cp ../em/charges.txt ./", 'Copying charge file from em folder')
out = runbash("gmx grompp -maxwarn 1 -f nvt.mdp -c em.gro -r em.gro -p complex.top -o nvt.tpr",
              "Compiling tpr file with grompp")

#out = runbash("gmx mdrun -v -deffnm nvt", 'Running NVT simulation')
out = !submit -i nvt.tpr --detach gromacs-2018.4-gmx mdrun -v -deffnm nvt
#!submit  gromacs-2018.4-gmx pdb2gmx -h
for line in out:
    print(line)
    
#out = runbash("echo 0 | gmx trjconv -f nvt.gro -s nvt.tpr -o nvt.pdb")
#for line in out:
#    print(line)